# Perceptron
## AND 
https://en.wikipedia.org/wiki/Perceptron

In [21]:
import numpy as np
X=[[0,0],[0,1],[1,0],[1,1]] # input
t=[0,0,0,1] # 실제값
# y 예측값

In [22]:
def perc(x):
    f=np.dot(w,x)+b
    if f>=0:
        y=1
    else:
        y=0
    return y

$$ w = w + \eta (t-y) x $$

In [59]:
w, b=[0,-1], 0 # 초기조건
eta = 0.1

In [60]:
for n in range(100):
    #error_sum = 0
    for i in range(4):
        error = t[i]- perc(X[i])
        #error_sum = error_sum + abs(error)
        
        w[0] = w[0] + eta * error * X[i][0]
        w[1] = w[1] + eta * error * X[i][1]
        b = b + eta * error
    
    #if(error_sum==0):
     #   break
w,b

([0.2, 0.19999999999999987], -0.20000000000000004)

In [61]:
perc(X[0]),perc(X[1]),perc(X[2]),perc(X[3])

(0, 0, 0, 1)

Perceptron:
$$ y_j = f ( \sum_i w_ix_i - b_j). $$

$$ w_i(t+1) = w_i(t) + \eta (t_j - y_j)x_{j,i} $$
* eta = learning rate
* t_j = target
* y_j = prediction

In [1]:
def perc(X):
    f=bias
    for i in range(2):
        f = f + w[i] * X[i]
    if f >= 0.0:
        y=1
    else:
        y=0
    return y

def train(X,y,l_rate,ne): # l_rate = learning rate / ne = number of epochs
    global bias
    for n in range(ne):
        sum_error = 0.0
        for sample, target in zip(X,y):
            yp=perc(sample)
            error = target - yp
            bias = bias + l_rate * error * x0

            for i in range(2):
                w[i]=w[i]+ l_rate*error*sample[i]
                sum_error += error**2 # 오차의 제곱

    return w

In [207]:
# data
x0 = 1
X = [[0,0],[0,1],[1,0],[1,1]]
y = [0,0,0,1]
w = [0,0]
bias = 0.0
l_rate = 0.1
ne=5

In [208]:
w = train(X,y,l_rate,ne)
w

[0.2, 0.1]

# XOR problem

In [66]:
X = [[0,0],[0,1],[1,0],[1,1]]
t = [0,1,1,0]

In [73]:
w, b=[0,0], 0 # 초기조건
eta = 0.1

In [76]:
for n in range(1000):
    for i in range(4):
        error = t[i]- perc(X[i])
        w[0] = w[0] + eta * error * X[i][0]
        w[1] = w[1] + eta * error * X[i][1]
        b = b + eta * error
    
w,b

([-0.1, 0.0], 0.0)

In [77]:
perc(X[0]),perc(X[1]),perc(X[2]),perc(X[3])

(1, 1, 0, 0)

# http://playground.tensorflow.org/